In [2]:
import duckdb
import pandas as pd

orders = pd.DataFrame({
    'customer_id': ['C001', 'C002', 'C001', 'C003', 'C002'],
    'order_date': ['2024-01-01', '2024-01-03', '2024-01-05', '2024-01-06', '2024-01-07'],
    'order_amount': [100, 200, 150, 300, 250]})

sales = pd.DataFrame({
    'product': ['Lipstick', 'Foundation', 'Lipstick', 'Blush', 'Foundation'],
    'revenue': [120, 150, 180, 90, 170],
    'date': ['2024-01-01', '2024-01-01', '2024-01-02', '2024-01-02', '2024-01-03']})

users = pd.DataFrame({
    'user_id': ['U001', 'U002', 'U003', 'U004'],
    'signup_date': ['2024-01-01', '2024-01-02', '2024-01-03', '2024-01-04'],
    'activity_date': ['2024-01-05', '2024-01-06', '2024-01-07', '2024-01-08']})

con = duckdb.connect()
con.register("orders", orders)
con.register("sales", sales)
con.register("users", users)


In [3]:
con.execute(""" select customer_id, order_date, order_amount
from ( select *, ROW_NUMBER() OVER (PARTITION BY customer_id ORDER BY order_date) AS rn from orders)
where rn = 1 """).df()

,customer_id,order_date,order_amount
0,C003,2024-01-06,300
1,C002,2024-01-03,200
2,C001,2024-01-01,100


In [9]:
con.execute(""" select product, date, revenue, rank() over (partition by date order by revenue desc) as rank
from sales """).df()

,product,date,revenue,rank
0,Foundation,2024-01-01,150,1
1,Lipstick,2024-01-01,120,2
2,Lipstick,2024-01-02,180,1
3,Blush,2024-01-02,90,2
4,Foundation,2024-01-03,170,1


In [10]:
con.execute(""" select date, product, revenue, LAG(revenue) over (partition by product order by date) as previous_day_revenue
from sales """).df()

,date,product,revenue,previous_day_revenue
0,2024-01-02,Blush,90,<NA>
1,2024-01-01,Foundation,150,<NA>
2,2024-01-03,Foundation,170,150
3,2024-01-01,Lipstick,120,<NA>
4,2024-01-02,Lipstick,180,120


In [12]:
con.execute(""" select customer_id, order_date, order_amount, sum(order_amount) over
(partition by customer_id order by order_date) as running_total from orders """).df()

,customer_id,order_date,order_amount,running_total
0,C002,2024-01-03,200,200.0
1,C002,2024-01-07,250,450.0
2,C003,2024-01-06,300,300.0
3,C001,2024-01-01,100,100.0
4,C001,2024-01-05,150,250.0


In [14]:
con.execute(""" select product, revenue, PERCENT_RANK() over (order by revenue desc) as percent_rank
from sales order by percent_rank""").df()

,product,revenue,percent_rank
0,Lipstick,180,0.00
1,Foundation,170,0.25
2,Foundation,150,0.50
3,Lipstick,120,0.75
4,Blush,90,1.00
